# Imports e Utilidades

In [2]:
import  warnings,                   \
        calendar,                   \
        pandas as pd,               \
        numpy as np,                \
        requests as rt,             \
        hydrobr as hbr,             \
        xml.etree.ElementTree as ET
from typing import List

from datetime import datetime, timedelta

from io import BytesIO

# Desativar as mensagens de 'warning' que ficam poluindo o output de alguns trechos de código.
warnings.filterwarnings("ignore")

In [3]:
def carregar_dados(file_name : str,
                   separator : str = "\t",
                   adjust : bool = True,
                   date_column : str = "ds"
                   ) -> pd.DataFrame:
    
    df = pd.read_csv(file_name, sep=separator, index_col=date_column, header=0, parse_dates=[date_column])

    if adjust:
        df = df.resample('D').first() # deixando a série contínua numa base diária

    # Deixando ajustado para usar com as libs Nixtla
    df['unique_id'] = 1
    df.reset_index(inplace=True)

    return df
# ============================================================================================ #
def get_telemetrica(codEstacao : str,
                    dataInicio : str,
                    dataFim : str,
                    save : bool = False) -> pd.DataFrame:
    # 1. Fazer a requisião ao servidor e pegar a árvore e a raiz dos dados 
    params = {'codEstacao':codEstacao, 'dataInicio':dataInicio, 'dataFim':dataFim}
    server = 'http://telemetriaws1.ana.gov.br/ServiceANA.asmx/DadosHidrometeorologicos'
    response = rt.get(server, params)
    tree = ET.ElementTree(ET.fromstring(response.content))
    root = tree.getroot()

    # 2. Iteração dentro dos elementos do XML procurando os dados que são disponibilizados para a estação
    list_vazao = []
    list_data = []
    list_cota = []
    list_chuva = []

    for i in root.iter('DadosHidrometereologicos'):

        data = i.find('DataHora').text
        try:
            vazao = float(i.find('Vazao').text)
        except TypeError:
            vazao = i.find('Vazao').text

        try:
            cota = float(i.find('Nivel').text)
        except TypeError:
            cota = i.find('Nivel').text

        try:
            chuva = float(i.find('Chuva').text)
        except TypeError:
            chuva = i.find('Chuva').text

        list_vazao.append(vazao)
        list_data.append(data)
        list_cota.append(cota)
        list_chuva.append(chuva)

    df = pd.DataFrame([list_data, list_cota, list_chuva, list_vazao]).transpose()
    df.columns = ['Data', 'Cota', 'Chuva', 'Vazao']
    df = df.sort_values(by='Data')
    df = df.set_index('Data')
    
    if save == True:
        df.to_excel(codEstacao+'_dados_tele.xlsx')
    
    return df
# ============================================================================================ #
def get_convencional(codEstacao : str,
                     dataInicio : str,
                     dataFim : str,
                     tipoDados : int,
                     nivelConsistencia : int,
                     save : bool = False) -> pd.DataFrame:
    """
        Série Histórica estação - HIDRO.
        codEstacao : Código Plu ou Flu
        dataInicio : <YYYY-mm-dd>
        dataFim : Caso não preenchido, trará até o último dado mais recente armazenado
        tipoDados : 1-Cotas, 2-Chuvas ou 3-Vazões
        nivelConsistencia : 1-Bruto ou 2-Consistido
    """

    # 1. Fazer a requisião ao servidor e pegar a árvore e a raiz dos dados 
    params = {'codEstacao':codEstacao, 'dataInicio':dataInicio, 'dataFim':dataFim,
              'tipoDados':tipoDados, 'nivelConsistencia':nivelConsistencia}
    
    server = 'http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroSerieHistorica'
    response = rt.get(server, params)
    tree = ET.ElementTree(ET.fromstring(response.content))
    root = tree.getroot()
    
    # 2. Iteração dentro dos elementos do XML procurando os dados que são disponibilizados para a estação
    list_data = []
    list_consistenciaF = []
    list_month_dates = []

    for i in root.iter('SerieHistorica'):

        consistencia = i.find('NivelConsistencia').text
        date = i.find('DataHora').text
        date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        last_day = calendar.monthrange(date.year, date.month)[1]
        month_dates = [date + timedelta(days=i) for i in range(last_day)]
        content = []
        list_consistencia = []

        for day in range(last_day):
            if tipoDados == 1:
                value = f'Cota{day+1:02d}'
            if tipoDados == 2:
                value = f'Chuva{day+1:02d}'
            if tipoDados == 3:
                value = f'Vazao{day+1:02d}'
            
            try:
                content.append(float(i.find(value).text))
                list_consistencia.append(int(consistencia))
            except TypeError:
                content.append(i.find(value).text)
                list_consistencia.append(int(consistencia))
            except AttributeError:
                content.append(None)
                list_consistencia.append(int(consistencia))
        
        list_data += content
        list_consistenciaF += list_consistencia
        list_month_dates += month_dates
    df = pd.DataFrame([list_month_dates, list_consistenciaF, list_data]).transpose()

    if tipoDados == 1:
        df.columns = ['Data','Consistencia','Cota']
    elif tipoDados == 2:
        df.columns = ['Data','Consistencia','Chuva']
    else: # Vazão
        df.columns = ['Data','Consistencia','Vazao']
    
    df = df.sort_values(by='Data')
    df = df.set_index('Data')

    if save == True:
        df.to_excel(codEstacao + '_dados_conv.xlsx')
    
    return df
# ============================================================================================ #
def gerar_dados_tele(estacao_principal : str,
                    outras_estacoes : List[str],
                    nome_arq : str,
                    dt_inicio : str,
                    dt_fim : str,
                    salvar : bool = False) -> None:
    """
            Este método vai pegar o código da 'estacao_principal' (que o usuário já sabe previamente que é uma telemétrica), baixar os dados da estação
        e concatenar (outer join) com os dados das outras estações telemétricas. Neste método já será realizada a conversão dos dados de 'object' para
        os tipos de acordo, ou seja, 'float' para os campos numéricos e 'datetime' para os campos de datahora.
            Como o desejo do trabalho é lidar com dados diários, já aproveita pra fazer a agregação dos dados desta maneira também.
            Após tudo isso, salva num arquivo xlsx para usos posteriores.

        Parâmetros:
            estacao_principal : str,
            outras_estacoes : List[str],
            nome_arq : str,
            dt_inicio : str = 'YYYY-mm-dd',
            dt_fim : str = 'YYYY-mm-dd',
            salvar : bool = True|False
    """

    df_result = get_telemetrica(codEstacao=estacao_principal, dataInicio=dt_inicio, dataFim=dt_fim)

    df_result.index = pd.to_datetime(df_result.index)
    df_result.Cota = pd.to_numeric(df_result.Cota, errors='coerce')
    df_result.Chuva = pd.to_numeric(df_result.Chuva, errors='coerce')
    df_result.Vazao = pd.to_numeric(df_result.Vazao, errors='coerce')

    df_result = df_result.resample('D').agg({'Cota': 'mean', 'Chuva': 'sum', 'Vazao': 'mean'})

    df_result.columns = ['t_ct_'+str(estacao_principal), 't_cv_'+str(estacao_principal), 't_vz_'+str(estacao_principal)]

    # Agora que já tenho os dados da estação que considero principal na análise (target)
    #   vou agregar com os dados das demais estações

    if outras_estacoes is not None:
        for e in outras_estacoes:
            df_temp = get_telemetrica(codEstacao=e, dataInicio=dt_inicio, dataFim=dt_fim)

            # Convertendo os dados
            df_temp.index = pd.to_datetime(df_temp.index)
            df_temp.Cota = pd.to_numeric(df_temp.Cota, errors='coerce')
            df_temp.Chuva = pd.to_numeric(df_temp.Chuva, errors='coerce')
            df_temp.Vazao = pd.to_numeric(df_temp.Vazao, errors='coerce')

            # Para as telemétricas já agrego aqui mesmo
            df_temp = df_temp.resample('D').agg({'Cota': 'mean', 'Chuva': 'sum', 'Vazao': 'mean'})

            # Ajeito os nomes das colunas pra conter de qual estacao os dado veio
            df_temp.columns = ['t_ct_'+e, 't_cv_'+e, 't_vz_'+e]

            df_result = pd.concat([df_result, df_temp], axis=1)

    if salvar:
        df_result.to_excel(nome_arq+'_dados_tele.xlsx')
# ============================================================================================ #
def gerar_dados_conv(estacao_principal : str,
                    outras_estacoes : List[str],
                    nome_arq : str,
                    dt_inicio : str,
                    dt_fim : str,
                    tp_dados : int,
                    nvl_consistencia : str,
                    drop_consistencia : bool = True, # Remover a coluna "NivelConsistência". Ela será irrelevante, até segunda ordem.
                    salvar : bool = False) -> None:
    """
            Este método vai pegar o código da 'estacao_principal' (que o usuário já sabe previamente que é uma convencional), baixar os dados da estação
        e concatenar (outer join) com os dados das outras estações convencionais. Neste método já será realizada a conversão dos dados de 'object' para
        os tipos de acordo, ou seja, 'float' para os campos numéricos e 'datetime' para os campos de datahora.
            Como o desejo do trabalho é lidar com dados diários, já aproveita pra fazer a agregação dos dados desta maneira também.
            Após tudo isso, salva num arquivo xlsx para usos posteriores.

        Parâmetros:
            estacao_principal : str,
            outras_estacoes : List[str],
            nome_arq : str,
            dt_inicio : str = 'YYYY-mm-dd',
            dt_fim : str = 'YYYY-mm-dd',
            tp_dados : int (1-cota | 2-chuva | 3-vazao),
            nvl_consistencia : int (1-bruto | 2-consistido),
            drop_consistencia : bool = True, (Remover a coluna "NivelConsistência". Ela será irrelevante, até segunda ordem)
            salvar : bool = False
    """

    df_result = get_convencional(codEstacao=estacao_principal, dataInicio=dt_inicio, dataFim=dt_fim, tipoDados=tp_dados, nivelConsistencia=nvl_consistencia)

    df_result.index = pd.to_datetime(df_result.index)

    if drop_consistencia:
        df_result.drop(columns=['Consistencia'], inplace=True)

    if tp_dados == 1:
        df_result.Cota = pd.to_numeric(df_result.Cota, errors='coerce')
        df_result = df_result.resample('D').agg({'Cota': 'mean'})
        df_result.columns = ['c_ct_'+str(estacao_principal)]
    elif tp_dados == 2:
        df_result.Chuva = pd.to_numeric(df_result.Chuva, errors='coerce')
        df_result = df_result.resample('D').agg({'Chuva': 'sum'})
        df_result.columns = ['c_cv_'+str(estacao_principal)]
    else: # Vazão
        df_result.Vazao = pd.to_numeric(df_result.Vazao, errors='coerce')
        df_result = df_result.resample('D').agg({'Vazao': 'mean'})
        df_result.columns = ['c_vz_'+str(estacao_principal)]

    # Agora que já tenho os dados da estação que considero principal na análise (target)
    #   vou agregar com os dados das demais estações

    for e in outras_estacoes:
        df_temp = get_convencional(codEstacao=e, dataInicio=dt_inicio, dataFim=dt_fim, tipoDados=tp_dados, nivelConsistencia=nvl_consistencia)

        # Convertendo os dados
        df_temp.index = pd.to_datetime(df_temp.index)

        if drop_consistencia:
            df_temp.drop(columns=['Consistencia'], inplace=True)

        if tp_dados == 1:
            df_temp.Cota = pd.to_numeric(df_temp.Cota, errors='coerce')
            df_temp = df_temp.resample('D').agg({'Cota': 'mean'})
            df_temp.columns = ['c_ct_'+str(e)]
        elif tp_dados == 2:
            df_temp.Chuva = pd.to_numeric(df_temp.Chuva, errors='coerce')
            df_temp = df_temp.resample('D').agg({'Chuva': 'sum'})
            df_temp.columns = ['c_cv_'+str(e)]
        else:
            df_temp.Vazao = pd.to_numeric(df_temp.Vazao, errors='coerce')
            df_temp = df_temp.resample('D').agg({'Vazao': 'mean'})
            df_temp.columns = ['c_vz_'+str(e)]

        df_result = pd.concat([df_result, df_temp], axis=1)

    if salvar:
        if tp_dados == 1:
            df_result.to_excel(nome_arq + '_dados_cota_conv.xlsx')
        elif tp_dados == 2:
            df_result.to_excel(nome_arq + '_dados_chuva_conv.xlsx')
        else:
            df_result.to_excel(nome_arq + '_dados_vazao_conv.xlsx')

# Download dos dados

# Baixo Rio Grande

In [16]:
# Baixando os dados das estações que serão utilizadas no trabalho
# As estações foram selecionadas a partir do sistema Data Rhama
# Aqui eu baixo os dados e salvo localmente
# >>>>>>>>>>>>> SÓ PRECISA FAZER ISSO UMA VEZ, POR ISSO O CÓDIGO FICA COMENTADO DEPOIS DE RODAR!!!! <<<<<<<<<<<<<

estacao_principal = '62020080'
outras_estacoes = ['61998080', '01950006', '02050001']

## Telemétricas

In [5]:
# Aplicando a lib HydroBR eu desejo saber se as estações em questão são do tipo convencional ou telemétrica
# O código não exclui o fato, eventual, de uma dada estação ser convencional E telemétrica, como é o caso aqui

lista_estacoes = hbr.get_data.ANA.list_telemetric() # Vendo primeiro se tem telemétrica
lista_estacoes.head()

4819it [00:03, 1372.91it/s]


,Name,Code,Status,SubBasin,City-State,Origem,Responsible,Elevation,Latitude,Longitude
1,RIO PRETO DA EVA,00259004,Ativo,15,RIO PRETO DA EVA-AM,Açudes Semiárido,00001 - ANA - Agência Nacional de Águas,0.00,-2.7003,-59.6997
2,UHE BELO MONTE BR230,00351004,Ativo,18,VITÓRIA DO XINGU-PA,Setor Elétrico,00594 - NORTE ENERGIA - Norte Energia S.A,33.00,-3.1267,-51.7906
3,UHE BELO MONTE SÍTIO PIMENTAL,00351005,Ativo,18,VITÓRIA DO XINGU-PA,Setor Elétrico,00594 - NORTE ENERGIA - Norte Energia S.A,110.00,-3.3758,-51.9403
4,UHE BELO MONTE VISTA ALEGRE,00352009,Ativo,18,ALTAMIRA-PA,Setor Elétrico,00594 - NORTE ENERGIA - Norte Energia S.A,125.00,-3.1186,-52.2525
5,UHE BELO MONTE SÃO FRANCISCO,00352010,Ativo,18,ALTAMIRA-PA,Setor Elétrico,00594 - NORTE ENERGIA - Norte Energia S.A,124.00,-3.2533,-52.3489


In [8]:
# Averiguando se as estações que tenho em mãos estão presentes neste conjunto de estações telemétricas

print("Estação {e} -> {p}".format(
                                e=estacao_principal,
                                p=(lista_estacoes['Code'] == estacao_principal).any()
                            )
)

# A estação principal em questão tem dados telemétricos

Estação 62020080 -> True


In [9]:
# Verificando as outras estações

for e in outras_estacoes:
    print("Estação {e} -> {p}".format(
                                        e=e,
                                        p=(lista_estacoes['Code'] == e).any()
                                    )
    )

# Estas estações também têm dados telemétricos

Estação 61998080 -> True
Estação 01950006 -> False
Estação 02050001 -> False


In [10]:
# Gerando um arquivo Excel com os dados das estações telemétricas

estacoes_tele = ['61998080']
gerar_dados_tele(
    estacao_principal=estacao_principal,
    outras_estacoes=estacoes_tele,
    nome_arq="baixo_rio_grande",
    dt_inicio='2013-01-01',
    dt_fim='2023-12-31',
    salvar=True
)

## Convencionais

### Cota/Vazão

In [11]:
# Aplicando a lib HydroBR eu desejo saber se as estações em questão são do tipo convencional ou telemétrica
# O código não exclui o fato, eventual, de uma dada estação ser convencional E telemétrica

lista_estacoes = hbr.get_data.ANA.list_flow( # Verificando se tem estações de cota/vazão primeiro
    state='MINAS GERAIS',
    source='ANA'
)

lista_estacoes.head()

2758it [00:01, 1701.20it/s]


,Name,Code,Type,DrainageArea,SubBasin,City,State,Responsible,Latitude,Longitude
1,SÃO ROQUE DE MINAS,40023000,1,None,40,SÃO ROQUE DE MINAS,MINAS GERAIS,IGAM-MG,-20.3344,-46.4697
2,VARGEM BONITA,40025000,1,299,40,VARGEM BONITA,MINAS GERAIS,ANA,-20.3272,-46.3661
3,IGUATAMA,40027000,1,None,40,IGUATAMA,MINAS GERAIS,IGAM-MG,-20.1717,-45.7261
4,FAZENDA DA BARCA,40030000,1,725,40,SÃO ROQUE DE MINAS,MINAS GERAIS,ANA,-20.1000,-46.3167
5,FAZENDA SAMBURÁ,40032000,1,754,40,SÃO ROQUE DE MINAS,MINAS GERAIS,ANA,-20.1508,-46.3033


In [17]:
# Averiguando se as estações que tenho em mãos estão presentes neste conjunto de estações convencionais de cota/vazão

print(
    "Estação {e} -> {p}".format(
        e=estacao_principal,
        p=(lista_estacoes['Code'] == estacao_principal).any()
    )
)

# A estação principal tem dados convencionais de cota/vazão

Estação 62020080 -> False


In [18]:
# Verificando as outras estações

for e in outras_estacoes:
    print(
        "Estação {e} -> {p}".format(
            e=e,
            p=(lista_estacoes['Code'] == e).any()
        )
    )

# Estas estações também têm dados convencionais de cota/vazão

Estação 61998080 -> False
Estação 01950006 -> False
Estação 02050001 -> False


In [45]:
# Gerando um arquivo Excel com os dados das estações convencionais

# SEM DADOS CONVENCIONAIS DE COTA/VAZAO

# estacoes_conv = [""]
# gerar_dados_conv(
#     estacao_principal=estacao_principal,
#     outras_estacoes=estacoes_conv,
#     nome_arq="baixo_rio_grande",
#     dt_inicio='2013-01-01',
#     dt_fim='2023-12-31',
#     tp_dados=1, # Cota
#     nvl_consistencia='2', # dados consistidos
#     salvar=True
# )

In [46]:
# Gerando um arquivo Excel com os dados das estações convencionais

# gerar_dados_conv(
#     estacao_principal=estacao_principal,
#     outras_estacoes=estacoes_conv,
#     nome_arq="baixo_rio_grande",
#     dt_inicio='2013-01-01',
#     dt_fim='2023-12-31',
#     tp_dados=3, # Vazão
#     nvl_consistencia='2', # dados consistidos
#     salvar=True
# )

### Chuva

In [19]:
lista_estacoes = hbr.get_data.ANA.list_prec(
    state='MINAS GERAIS',
    source='ANA'
)

lista_estacoes.head()

2434it [00:01, 1812.37it/s]


,Name,Code,Type,SubBasin,City,State,Responsible,Latitude,Longitude
1,AÇUDE DO ESTREITO,01442019,2,44,ESPINOSA,MINAS GERAIS,DNOCS,-14.8167,-42.8000
2,ESPIGÃO,01442020,2,44,ESPINOSA,MINAS GERAIS,SUDENE,-14.9833,-42.5667
3,ITAMIRIM,01442021,2,44,ESPINOSA,MINAS GERAIS,SUDENE,-14.7667,-42.8833
4,ESPINOSA,01442022,2,44,ESPINOSA,MINAS GERAIS,SUDENE,-14.9333,-42.8167
5,ESPINOSA,01442025,2,44,ESPINOSA,MINAS GERAIS,INMET,-14.9333,-42.8167


In [22]:
print(
    "Estação {e} -> {p}".format(
        e=estacao_principal,
        p=(lista_estacoes['Code'] == estacao_principal).any()
    )
)

# A estação principal NÃO tem dados convencionais de chuva

Estação 62020080 -> False


In [23]:
# Verificando as outras estações

for e in outras_estacoes:
    print(
        "Estação {e} -> {p}".format(
            e=e,
            p=(lista_estacoes['Code'] == e).any()
        )
    )

# Estas estações têm dados convencionais de chuva

Estação 61998080 -> False
Estação 01950006 -> False
Estação 02050001 -> False


In [24]:
# Gerando um arquivo Excel com os dados das estações convencionais

# SEM DADOS CONVENCIONAIS DE CHUVA

# estacoes_conv = [""]
# gerar_dados_conv(
#     estacao_principal=estacao_principal,
#     outras_estacoes=estacoes_conv,
#     nome_arq="baixo_rio_jequitinhonha",
#     dt_inicio='2013-01-01',
#     dt_fim='2023-12-31',
#     tp_dados=2, # Chuva
#     nvl_consistencia='2', # dados consistidos
#     salvar=True
# )

## Juntando os dados

In [25]:
arquivos = ["baixo_rio_grande_dados_tele.xlsx"]

In [26]:
# Vou fazer a carga primeiro dos dados telemétricos, porque é onde tem mais informação de uma única vez.
# Depois concateno os outros arquivos. Mas a ordem tanto faz aqui, só estipulei assim porque acho melhor

df = pd.read_excel(arquivos[0], sheet_name=0, index_col=0, header=0, parse_dates=['Data'])

for a in range(1, len(arquivos)):
    df_temp = pd.read_excel(arquivos[a], sheet_name=0, index_col=0, header=0, parse_dates=['Data'])
    df = pd.concat([df, df_temp], axis=1)

df

,t_ct_62020080,t_cv_62020080,t_vz_62020080,t_ct_61998080,t_cv_61998080,t_vz_61998080
Data,,,,,,
2014-12-30,32037.583333,0,NaN,NaN,NaN,NaN
2014-12-31,NaN,0,NaN,NaN,NaN,NaN
2015-01-01,NaN,0,NaN,NaN,NaN,NaN
2015-01-02,NaN,0,NaN,NaN,NaN,NaN
2015-01-03,NaN,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2023-12-27,32592.583333,0,3252.283333,37878.250000,0.0,1874.664000
2023-12-28,32594.750000,0,3698.487500,37879.041667,0.0,1542.200000
2023-12-29,32593.833333,0,3620.600000,37873.489583,0.0,2566.627586


In [28]:
df.columns

Index(['t_ct_62020080', 't_cv_62020080', 't_vz_62020080', 't_ct_61998080',
       't_cv_61998080', 't_vz_61998080'],
      dtype='object')

In [29]:
# Vou remover as colunas das cotas

df = df.drop(columns=["t_ct_62020080", "t_ct_61998080"])
df

,t_cv_62020080,t_vz_62020080,t_cv_61998080,t_vz_61998080
Data,,,,
2014-12-30,0,NaN,NaN,NaN
2014-12-31,0,NaN,NaN,NaN
2015-01-01,0,NaN,NaN,NaN
2015-01-02,0,NaN,NaN,NaN
2015-01-03,0,NaN,NaN,NaN
...,...,...,...,...
2023-12-27,0,3252.283333,0.0,1874.664000
2023-12-28,0,3698.487500,0.0,1542.200000
2023-12-29,0,3620.600000,0.0,2566.627586


In [30]:
df.columns

Index(['t_cv_62020080', 't_vz_62020080', 't_cv_61998080', 't_vz_61998080'], dtype='object')

In [33]:
# Fazendo o merge das colunas de vazão que são correspondentes à mesma estação
# Acontece que existem gaps entre os dados, o que é estranho, porque a estação telemétrica tem dados que a convencional não tem e vice-versa.
# Vou contar qual coluna tem mais dados e depois executar um 'fillna'

colunas_esquerda = ['t_cv_62020080', 't_vz_62020080']
colunas_direita = ['t_cv_61998080', 't_vz_61998080']

for i, j in zip(colunas_esquerda, colunas_direita):
    print(i, j, df[i].isna().sum(), df[j].isna().sum())

# A coluna que tiver menos buracos será preenchida com os dados da coluna que tem mais dados faltantes

t_cv_62020080 t_cv_61998080 0 169
t_vz_62020080 t_vz_61998080 2099 3181


In [32]:
# # Preenchendo a coluna que tem menos dados faltantes com a outra correspondente
# # Fazer isso para cada coluna, contudo, tem colunas que tem dados faltando demais. Neste caso, darei drop nelas inteiramente.
# df['c_vz_56994500'].fillna(df['t_vz_56994500'], inplace=True)
# df['t_vz_56990850'].fillna(df['c_vz_56990850'], inplace=True)
# df['t_vz_56990005'].fillna(df['c_vz_56990005'], inplace=True)

# df['c_vz_56994500'].isna().sum(), df['t_vz_56990850'].isna().sum(), df['t_vz_56990005'].isna().sum()

In [34]:
# Verificando a quantidade de valores "NaN" em cada coluna

print("Quantidade de NaN por coluna")
print(df.isna().sum())

print("Percentual de NaN por coluna")
for c in np.asarray(df.columns):
    print(c, (df[c].isna().sum()/len(df))*100)

Quantidade de NaN por coluna
t_cv_62020080       0
t_vz_62020080    2099
t_cv_61998080     169
t_vz_61998080    3181
dtype: int64
Percentual de NaN por coluna
t_cv_62020080 0.0
t_vz_62020080 63.81878990574643
t_cv_61998080 5.138339920948617
t_vz_61998080 96.71632715110977


In [35]:
# Esta coluna está quase completamente vazia:
# t_vz_61998080
# Vou remover

df = df.drop(columns=["t_vz_61998080"])
df

,t_cv_62020080,t_vz_62020080,t_cv_61998080
Data,,,
2014-12-30,0,NaN,NaN
2014-12-31,0,NaN,NaN
2015-01-01,0,NaN,NaN
2015-01-02,0,NaN,NaN
2015-01-03,0,NaN,NaN
...,...,...,...
2023-12-27,0,3252.283333,0.0
2023-12-28,0,3698.487500,0.0
2023-12-29,0,3620.600000,0.0


In [36]:
# Deixando os dados contínuos, numa base diária.
df = df.resample('D').first()
df

,t_cv_62020080,t_vz_62020080,t_cv_61998080
Data,,,
2014-12-30,0,NaN,NaN
2014-12-31,0,NaN,NaN
2015-01-01,0,NaN,NaN
2015-01-02,0,NaN,NaN
2015-01-03,0,NaN,NaN
...,...,...,...
2023-12-27,0,3252.283333,0.0
2023-12-28,0,3698.487500,0.0
2023-12-29,0,3620.600000,0.0


In [37]:
# Neste momento, tenho o DataFrame com os dados EXATAMENTE da forma que preciso.
# Posso, inclusive, exportar isso para um arquivo de Excel
# É o que farei, pois se precisar retornar aos dados originais, será mais fácil que fazer toda engenharia até aqui

df.to_excel('./arquivo_final/baixo_rio_grande_final.xlsx')